# Databases — Overview

## Purpose
This section provides a comprehensive foundation in database technologies, covering fundamental concepts, different database paradigms, and the trade-offs involved in choosing the right database for your application. Understanding databases is crucial for building scalable, reliable, and performant applications.

## Key Questions
- What is a database and why do we need them?
- What are ACID and BASE properties, and when should each be prioritized?
- How does the CAP theorem influence database design decisions?
- What are the differences between SQL and NoSQL databases?
- How do you choose the right database for a specific use case?

## Topics Covered
| Topic | Description |
|-------|-------------|
| **Database Fundamentals** | What databases are, types, and core concepts |
| **ACID Properties** | Atomicity, Consistency, Isolation, Durability |
| **BASE Properties** | Basically Available, Soft state, Eventually consistent |
| **CAP Theorem** | Consistency, Availability, Partition tolerance trade-offs |
| **SQL Databases** | Relational databases, schemas, normalization |
| **NoSQL Databases** | Document, Key-Value, Column-family, Graph databases |
| **Database Selection** | Choosing the right database for your use case |

In [ ]:
# Import required libraries
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

---
## 1. What is a Database?

A **database** is an organized collection of structured data stored electronically. Databases are managed by **Database Management Systems (DBMS)** that provide mechanisms for:

- **Data Storage**: Efficiently storing large amounts of data
- **Data Retrieval**: Quickly querying and accessing data
- **Data Integrity**: Ensuring data accuracy and consistency
- **Concurrency Control**: Managing simultaneous access by multiple users
- **Security**: Protecting data from unauthorized access
- **Backup & Recovery**: Protecting against data loss

### Why Databases Matter

| Without Database | With Database |
|-----------------|---------------|
| Data stored in flat files | Structured, organized storage |
| No query language | Powerful query capabilities |
| Manual concurrency handling | Built-in transaction support |
| Data redundancy issues | Normalization reduces redundancy |
| Limited scalability | Designed for scale |

In [ ]:
# Visualization: Database Types Hierarchy
db_types = {
    'Database Types': ['Relational (SQL)', 'Non-Relational (NoSQL)', 'NewSQL', 'Time-Series', 'Graph'],
    'Examples': ['PostgreSQL, MySQL, Oracle', 'MongoDB, Cassandra, Redis', 'CockroachDB, TiDB', 'InfluxDB, TimescaleDB', 'Neo4j, Amazon Neptune'],
    'Best For': ['Structured data, ACID compliance', 'Flexible schemas, horizontal scaling', 'SQL + horizontal scaling', 'IoT, metrics, logs', 'Connected data, relationships'],
    'Market Share (%)': [60, 25, 5, 5, 5]
}

df_types = pd.DataFrame(db_types)

fig = go.Figure(data=[go.Pie(
    labels=df_types['Database Types'],
    values=df_types['Market Share (%)'],
    hole=0.4,
    textinfo='label+percent',
    marker=dict(colors=px.colors.qualitative.Set2)
)])

fig.update_layout(
    title='Database Types Market Distribution',
    annotations=[dict(text='DB Types', x=0.5, y=0.5, font_size=16, showarrow=False)],
    height=450
)
fig

---
## 2. ACID Properties

**ACID** properties are fundamental guarantees that ensure reliable transaction processing in relational databases:

### Atomicity
- Transactions are "all or nothing"
- If any part fails, the entire transaction is rolled back
- Example: Bank transfer—both debit and credit must succeed

### Consistency
- Database moves from one valid state to another
- All constraints, triggers, and rules are enforced
- Example: Account balance cannot be negative

### Isolation
- Concurrent transactions don't interfere with each other
- Each transaction appears to run in isolation
- Isolation levels: Read Uncommitted → Read Committed → Repeatable Read → Serializable

### Durability
- Once committed, data persists even after system failure
- Typically achieved through write-ahead logging (WAL)
- Example: After transfer confirmation, data survives power outage

In [ ]:
# ACID Properties Visualization
acid_data = {
    'Property': ['Atomicity', 'Consistency', 'Isolation', 'Durability'],
    'Description': [
        'All or Nothing',
        'Valid State Transitions',
        'No Interference',
        'Data Persists'
    ],
    'Mechanism': [
        'Rollback on failure',
        'Constraints & Rules',
        'Locking & MVCC',
        'Write-Ahead Logging'
    ],
    'Importance': [95, 90, 85, 98]
}

fig = go.Figure()

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

fig.add_trace(go.Bar(
    x=acid_data['Property'],
    y=acid_data['Importance'],
    marker_color=colors,
    text=[f"{acid_data['Description'][i]}<br><i>{acid_data['Mechanism'][i]}</i>" 
          for i in range(4)],
    textposition='outside',
    hovertemplate='<b>%{x}</b><br>Importance: %{y}%<extra></extra>'
))

fig.update_layout(
    title='ACID Properties — Core Guarantees of Relational Databases',
    xaxis_title='Property',
    yaxis_title='Importance Score',
    yaxis_range=[0, 120],
    height=450,
    showlegend=False
)
fig

In [ ]:
# Example: ACID Transaction in SQL (Pseudocode)
acid_transaction_example = """
-- ACID Transaction Example: Bank Transfer
BEGIN TRANSACTION;

-- Check if source account has sufficient balance (Consistency)
SELECT balance FROM accounts WHERE account_id = 'A123' FOR UPDATE;

-- Debit from source account
UPDATE accounts 
SET balance = balance - 500.00 
WHERE account_id = 'A123';

-- Credit to destination account  
UPDATE accounts 
SET balance = balance + 500.00 
WHERE account_id = 'B456';

-- Log the transaction
INSERT INTO transaction_log (from_account, to_account, amount, timestamp)
VALUES ('A123', 'B456', 500.00, NOW());

COMMIT;  -- Atomicity: All changes are persisted together
         -- Durability: Data is now safe on disk

-- If any step fails → ROLLBACK (Atomicity)
-- Concurrent transfers don't see partial state (Isolation)
"""
print(acid_transaction_example)

---
## 3. BASE Properties

**BASE** is an alternative approach favored by many NoSQL databases, trading strict consistency for availability and performance:

### Basically Available
- System guarantees availability (responses, even if stale)
- Prioritizes uptime over perfect consistency
- Uses replication and distribution for fault tolerance

### Soft State
- State of the system may change over time
- Even without input, due to eventual consistency
- Data might be temporarily inconsistent across replicas

### Eventually Consistent
- System will become consistent over time
- If no new updates, all reads will eventually return the last updated value
- Trade-off: Faster writes, eventual synchronization

In [ ]:
# ACID vs BASE Comparison
comparison_data = {
    'Aspect': ['Consistency Model', 'Availability', 'Scalability', 
               'Performance', 'Use Case', 'Complexity'],
    'ACID': [10, 6, 5, 6, 'Financial, Critical', 7],
    'BASE': [6, 10, 10, 9, 'Social, Analytics', 8]
}

df_compare = pd.DataFrame(comparison_data)

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=df_compare['ACID'],
    theta=df_compare['Aspect'],
    fill='toself',
    name='ACID',
    line_color='#FF6B6B',
    fillcolor='rgba(255, 107, 107, 0.3)'
))

fig.add_trace(go.Scatterpolar(
    r=df_compare['BASE'],
    theta=df_compare['Aspect'],
    fill='toself',
    name='BASE',
    line_color='#4ECDC4',
    fillcolor='rgba(78, 205, 196, 0.3)'
))

fig.update_layout(
    polar=dict(
        radialaxis=dict(visible=True, range=[0, 10])
    ),
    title='ACID vs BASE — Property Comparison',
    height=500,
    showlegend=True
)
fig

---
## 4. CAP Theorem

The **CAP Theorem** (Brewer's Theorem) states that a distributed data store can only provide **two out of three** guarantees:

### Consistency (C)
- Every read receives the most recent write or an error
- All nodes see the same data at the same time

### Availability (A)
- Every request receives a response (success or failure)
- System remains operational at all times

### Partition Tolerance (P)
- System continues to operate despite network partitions
- Nodes can't communicate but system still functions

### The Trade-off
In practice, **network partitions are inevitable** in distributed systems, so you must choose between:
- **CP**: Sacrifice availability for consistency (e.g., HBase, MongoDB)
- **AP**: Sacrifice consistency for availability (e.g., Cassandra, DynamoDB)

In [ ]:
# CAP Theorem Triangle Visualization
import numpy as np

# Create triangle vertices
vertices = {
    'C': (0, 1),      # Consistency (top)
    'A': (-0.866, -0.5),  # Availability (bottom-left)
    'P': (0.866, -0.5)    # Partition Tolerance (bottom-right)
}

# Database positions (approximate based on their characteristics)
databases = {
    'PostgreSQL': {'pos': (-0.1, 0.6), 'type': 'CA', 'color': '#336791'},
    'MySQL': {'pos': (0.1, 0.55), 'type': 'CA', 'color': '#00758F'},
    'MongoDB': {'pos': (0.4, 0.1), 'type': 'CP', 'color': '#4DB33D'},
    'HBase': {'pos': (0.5, 0.0), 'type': 'CP', 'color': '#D22128'},
    'Redis': {'pos': (0.3, 0.2), 'type': 'CP', 'color': '#DC382D'},
    'Cassandra': {'pos': (0.0, -0.3), 'type': 'AP', 'color': '#1287B1'},
    'DynamoDB': {'pos': (-0.2, -0.25), 'type': 'AP', 'color': '#FF9900'},
    'CouchDB': {'pos': (0.15, -0.35), 'type': 'AP', 'color': '#E42528'},
    'Riak': {'pos': (-0.1, -0.4), 'type': 'AP', 'color': '#FFA500'}
}

fig = go.Figure()

# Draw triangle
triangle_x = [vertices['C'][0], vertices['A'][0], vertices['P'][0], vertices['C'][0]]
triangle_y = [vertices['C'][1], vertices['A'][1], vertices['P'][1], vertices['C'][1]]

fig.add_trace(go.Scatter(
    x=triangle_x,
    y=triangle_y,
    mode='lines',
    line=dict(color='gray', width=2),
    showlegend=False,
    hoverinfo='skip'
))

# Add vertex labels
for label, pos in vertices.items():
    full_label = {'C': 'Consistency', 'A': 'Availability', 'P': 'Partition Tolerance'}[label]
    offset_y = 0.15 if label == 'C' else -0.15
    fig.add_annotation(
        x=pos[0], y=pos[1] + offset_y,
        text=f"<b>{full_label}</b>",
        showarrow=False,
        font=dict(size=14)
    )

# Add databases
for db_name, db_info in databases.items():
    fig.add_trace(go.Scatter(
        x=[db_info['pos'][0]],
        y=[db_info['pos'][1]],
        mode='markers+text',
        marker=dict(size=20, color=db_info['color']),
        text=[db_name],
        textposition='top center',
        name=f"{db_name} ({db_info['type']})",
        hovertemplate=f"<b>{db_name}</b><br>Type: {db_info['type']}<extra></extra>"
    ))

# Add region labels
fig.add_annotation(x=0.5, y=0.3, text="<b>CP</b><br>Consistent & Partition-tolerant", 
                   showarrow=False, font=dict(size=10, color='gray'))
fig.add_annotation(x=-0.5, y=0.3, text="<b>CA</b><br>Consistent & Available", 
                   showarrow=False, font=dict(size=10, color='gray'))
fig.add_annotation(x=0, y=-0.55, text="<b>AP</b><br>Available & Partition-tolerant", 
                   showarrow=False, font=dict(size=10, color='gray'))

fig.update_layout(
    title='CAP Theorem — Database Classification',
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[-1.2, 1.2]),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[-0.8, 1.3]),
    height=600,
    showlegend=True,
    legend=dict(x=1.02, y=1)
)
fig

---
## 5. SQL vs NoSQL Databases

### SQL (Relational) Databases

**Characteristics:**
- Structured data with predefined schemas
- Tables with rows and columns
- Strong ACID compliance
- SQL query language
- Vertical scaling (scale up)

**Examples:** PostgreSQL, MySQL, Oracle, SQL Server, SQLite

### NoSQL (Non-Relational) Databases

| Type | Description | Examples | Best For |
|------|-------------|----------|----------|
| **Document** | JSON-like documents | MongoDB, CouchDB | Content management, catalogs |
| **Key-Value** | Simple key-value pairs | Redis, DynamoDB | Caching, sessions |
| **Column-Family** | Column-oriented storage | Cassandra, HBase | Analytics, time-series |
| **Graph** | Nodes and relationships | Neo4j, Neptune | Social networks, recommendations |

In [ ]:
# SQL vs NoSQL Feature Comparison
features = [
    'Schema Flexibility',
    'Horizontal Scaling',
    'Query Complexity',
    'ACID Compliance',
    'Read Performance',
    'Write Performance',
    'Data Relationships',
    'Learning Curve'
]

sql_scores = [3, 4, 9, 10, 7, 6, 9, 7]
nosql_scores = [10, 9, 5, 4, 9, 9, 4, 6]

fig = go.Figure()

fig.add_trace(go.Bar(
    name='SQL Databases',
    x=features,
    y=sql_scores,
    marker_color='#336791',
    text=sql_scores,
    textposition='outside'
))

fig.add_trace(go.Bar(
    name='NoSQL Databases',
    x=features,
    y=nosql_scores,
    marker_color='#4DB33D',
    text=nosql_scores,
    textposition='outside'
))

fig.update_layout(
    title='SQL vs NoSQL — Feature Comparison (Score out of 10)',
    xaxis_title='Feature',
    yaxis_title='Score',
    barmode='group',
    height=500,
    yaxis_range=[0, 12],
    xaxis_tickangle=-45,
    legend=dict(x=0.7, y=1.1, orientation='h')
)
fig

In [ ]:
# Data Model Comparison Examples
sql_example = """
┌─────────────────────────────────────────────────────────────┐
│              SQL (Relational) Data Model                     │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   users table              orders table                     │
│   ┌────┬───────┬────────┐  ┌────┬─────────┬────────┬─────┐  │
│   │ id │ name  │ email  │  │ id │ user_id │ amount │date │  │
│   ├────┼───────┼────────┤  ├────┼─────────┼────────┼─────┤  │
│   │ 1  │ Alice │ a@x.com│  │ 1  │    1    │ 99.99  │ ... │  │
│   │ 2  │ Bob   │ b@x.com│  │ 2  │    1    │ 49.99  │ ... │  │
│   └────┴───────┴────────┘  │ 3  │    2    │ 29.99  │ ... │  │
│                            └────┴─────────┴────────┴─────┘  │
│                                                             │
│   Query: SELECT u.name, SUM(o.amount)                       │
│          FROM users u JOIN orders o ON u.id = o.user_id     │
│          GROUP BY u.id;                                     │
└─────────────────────────────────────────────────────────────┘
"""

nosql_example = """
┌─────────────────────────────────────────────────────────────┐
│           NoSQL (Document) Data Model - MongoDB             │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   {                                                         │
│     "_id": ObjectId("..."),                                 │
│     "name": "Alice",                                        │
│     "email": "a@x.com",                                     │
│     "orders": [                   ← Embedded documents      │
│       { "amount": 99.99, "date": "2024-01-15" },            │
│       { "amount": 49.99, "date": "2024-01-20" }             │
│     ],                                                      │
│     "total_spent": 149.98          ← Denormalized field     │
│   }                                                         │
│                                                             │
│   Query: db.users.find({ name: "Alice" })                   │
│   → Returns user with all orders in single query            │
└─────────────────────────────────────────────────────────────┘
"""

print(sql_example)
print(nosql_example)

---
## 6. When to Use Which Database Type

### Decision Framework

Choosing the right database depends on several factors:

1. **Data Structure**: Is your data highly structured or flexible?
2. **Consistency Requirements**: Do you need strong ACID guarantees?
3. **Scale Requirements**: Horizontal vs vertical scaling needs?
4. **Query Patterns**: Complex joins vs simple lookups?
5. **Team Expertise**: SQL knowledge vs NoSQL experience?

In [ ]:
# Database Selection Decision Matrix
use_cases = {
    'Use Case': [
        'E-commerce Transactions',
        'User Sessions/Cache',
        'Content Management',
        'Social Network',
        'IoT/Time-Series Data',
        'Financial Trading',
        'Real-time Analytics',
        'Recommendation Engine'
    ],
    'Best Database Type': [
        'SQL (PostgreSQL)',
        'Key-Value (Redis)',
        'Document (MongoDB)',
        'Graph (Neo4j)',
        'Time-Series (InfluxDB)',
        'SQL (PostgreSQL)',
        'Column-Family (Cassandra)',
        'Graph (Neo4j)'
    ],
    'Priority': [
        'ACID Compliance',
        'Speed',
        'Flexibility',
        'Relationships',
        'Write Speed',
        'Consistency',
        'Scale',
        'Relationships'
    ],
    'Consistency Need': [10, 3, 5, 6, 4, 10, 5, 6],
    'Scale Need': [6, 8, 7, 7, 10, 5, 10, 7]
}

df_use_cases = pd.DataFrame(use_cases)

fig = px.scatter(
    df_use_cases,
    x='Consistency Need',
    y='Scale Need',
    color='Best Database Type',
    size=[40]*8,
    text='Use Case',
    hover_data=['Priority'],
    color_discrete_sequence=px.colors.qualitative.Set1
)

fig.update_traces(textposition='top center', textfont_size=9)

fig.update_layout(
    title='Database Selection by Use Case — Consistency vs Scale Requirements',
    xaxis_title='Consistency Requirement (1-10)',
    yaxis_title='Scale Requirement (1-10)',
    height=550,
    xaxis_range=[0, 12],
    yaxis_range=[0, 12]
)

# Add quadrant labels
fig.add_annotation(x=2, y=9, text="<b>AP Systems</b><br>High Scale, Low Consistency",
                   showarrow=False, font=dict(size=10, color='gray'))
fig.add_annotation(x=9, y=9, text="<b>NewSQL</b><br>High Scale, High Consistency",
                   showarrow=False, font=dict(size=10, color='gray'))
fig.add_annotation(x=9, y=2, text="<b>Traditional SQL</b><br>Low Scale, High Consistency",
                   showarrow=False, font=dict(size=10, color='gray'))

fig

In [ ]:
# Decision Tree Visualization
decision_guide = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                    DATABASE SELECTION DECISION GUIDE                          ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌─────────────────────────────────────────────┐                             ║
║  │ Do you need ACID transactions?              │                             ║
║  └─────────────────┬───────────────────────────┘                             ║
║                    │                                                         ║
║        ┌───────────┴───────────┐                                             ║
║        ▼                       ▼                                             ║
║  ┌─────────────┐         ┌─────────────┐                                     ║
║  │    YES      │         │     NO      │                                     ║
║  └──────┬──────┘         └──────┬──────┘                                     ║
║         │                       │                                            ║
║         ▼                       ▼                                            ║
║  ┌─────────────────┐     ┌─────────────────────────┐                         ║
║  │ Need horizontal │     │ What's your data model? │                         ║
║  │ scaling?        │     └────────────┬────────────┘                         ║
║  └────────┬────────┘                  │                                      ║
║           │                ┌──────────┼──────────┬──────────┐                ║
║    ┌──────┴──────┐         ▼          ▼          ▼          ▼                ║
║    ▼             ▼      Key-Value  Document  Graph    Time-Series            ║
║   YES           NO         │          │         │          │                 ║
║    │             │         ▼          ▼         ▼          ▼                 ║
║    ▼             ▼       Redis     MongoDB    Neo4j    InfluxDB              ║
║  NewSQL       SQL DB   DynamoDB   CouchDB   Neptune  TimescaleDB             ║
║ (CockroachDB) (PostgreSQL)                                                   ║
║ (TiDB)        (MySQL)                                                        ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""
print(decision_guide)

In [ ]:
# Popular Databases Summary Table
databases_summary = {
    'Database': ['PostgreSQL', 'MySQL', 'MongoDB', 'Redis', 'Cassandra', 
                 'Neo4j', 'DynamoDB', 'InfluxDB', 'CockroachDB'],
    'Type': ['SQL', 'SQL', 'Document', 'Key-Value', 'Column-Family',
             'Graph', 'Key-Value', 'Time-Series', 'NewSQL'],
    'CAP': ['CA', 'CA', 'CP', 'CP', 'AP', 'CA', 'AP', 'AP', 'CP'],
    'ACID': ['Yes', 'Yes', 'Partial', 'No', 'Tunable', 'Yes', 'Yes', 'No', 'Yes'],
    'Scaling': ['Vertical', 'Vertical', 'Horizontal', 'Horizontal', 'Horizontal',
                'Vertical', 'Horizontal', 'Horizontal', 'Horizontal'],
    'Best Use Case': [
        'General purpose, complex queries',
        'Web applications, read-heavy',
        'Flexible schemas, rapid development',
        'Caching, sessions, real-time',
        'Write-heavy, time-series at scale',
        'Relationships, recommendations',
        'Serverless, variable traffic',
        'Metrics, IoT, monitoring',
        'Distributed SQL, global scale'
    ]
}

df_summary = pd.DataFrame(databases_summary)

fig = go.Figure(data=[go.Table(
    header=dict(
        values=list(df_summary.columns),
        fill_color='#336791',
        font=dict(color='white', size=12),
        align='left',
        height=35
    ),
    cells=dict(
        values=[df_summary[col] for col in df_summary.columns],
        fill_color=[['#f0f0f0', 'white']*5],
        align='left',
        height=28,
        font=dict(size=11)
    )
)])

fig.update_layout(
    title='Popular Database Technologies — Quick Reference',
    height=400
)
fig

---
## 7. Key Concepts Summary

### ACID vs BASE at a Glance

| Property | ACID | BASE |
|----------|------|------|
| **Focus** | Strong consistency | High availability |
| **Guarantee** | Immediate consistency | Eventual consistency |
| **Trade-off** | Lower availability | Temporary inconsistency |
| **Best for** | Financial, inventory | Social media, analytics |
| **Examples** | PostgreSQL, MySQL | Cassandra, DynamoDB |

### CAP Theorem in Practice

| Combination | Behavior | Examples |
|-------------|----------|----------|
| **CP** | Consistent + Partition-tolerant | MongoDB, HBase, Redis |
| **AP** | Available + Partition-tolerant | Cassandra, DynamoDB, CouchDB |
| **CA** | Consistent + Available (single node) | PostgreSQL, MySQL (non-distributed) |

### SQL vs NoSQL Selection

**Choose SQL when:**
- Data is highly structured and relationships are important
- ACID compliance is critical
- Complex queries with JOINs are needed
- Data integrity is paramount

**Choose NoSQL when:**
- Schema flexibility is required
- Horizontal scaling is a priority
- High write throughput is needed
- Data model fits document/key-value/graph patterns

---
## Takeaway

### 🎯 Key Insights

1. **No One-Size-Fits-All**: The "best" database depends entirely on your use case, data model, and requirements.

2. **Understand the Trade-offs**: 
   - ACID gives you reliability at the cost of performance/scale
   - BASE gives you availability and scale at the cost of immediate consistency
   - CAP theorem means you can't have everything in distributed systems

3. **Start with Requirements**:
   - What are your consistency needs?
   - How will you scale?
   - What are your query patterns?
   - What does your data look like?

4. **Polyglot Persistence**: Modern applications often use multiple databases, each optimized for specific use cases.

5. **Evolution Over Revolution**: Many databases are evolving (e.g., PostgreSQL gaining NoSQL features, MongoDB adding ACID transactions). The lines are blurring.

### 📚 Next Steps

Explore the detailed sections in this database knowledge base:
- **Data Modeling**: Schema design, normalization, denormalization
- **Indexing & Query Optimization**: Making your queries fast
- **Replication & Sharding**: Scaling strategies
- **Transactions & Concurrency**: Managing concurrent access
- **Backup & Recovery**: Protecting your data